In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

In [5]:
method_list = ['AE'] #['ESD', 'CA', 'UCE', 'Ours']
prompt_name = 'short_niche_art_prompts'

method_list = ['AP+', 'AE'] #['ESD', 'CA', 'UCE', 'Ours']
prompt_name = 'long_niche_art_prompts'

temp_path1 = "evaluation_folder/artist/artist_{}_{}_lpips.csv" #"evaluation_folder/artist_{method}_{prompt_name}_lpips.csv"
# each row: 'erased_artist', 'prompt_artist' prompt, num, lpips score
# total row: len(art_list) * len(df) * num_samples
temp_path2 = "evaluation_folder/artist/artist_{}_{}_clip.csv" #"evaluation_folder/artist_{method}_{prompt_name}_clip.csv"
# each row: 'erased_artist', 'prompt_artist', prompt, num, score1: clip score with prompt, score2: clip score with artist, score3: clip score with prompt without artist
# total row: len(art_list) * len(df) * num_samples

SD_path = "evaluation_folder/artist/artist_SD_{}_clip.csv"#"evaluation_folder/artist_{method}_{prompt_name}_SD.csv"

In [6]:
# read SD file 
df_SD = pd.read_csv(SD_path.format(prompt_name))
df_SD2 = pd.read_csv('evaluation_folder/artist/artist_SD_vangogh_prompts_clip.csv')
# combine two dataframes
df_SD = pd.concat([df_SD, df_SD2])

data_dict = dict()
for _, row in df_SD.iterrows():

    prompt = row['prompt']
    prompt_artist = row['prompt_artist']
    erased_artist = row['erased_artist']
    score1 = row['score1']
    score2 = row['score2']
    score3 = row['score3']
    
    if prompt_artist not in data_dict:
        data_dict[prompt_artist] = dict()
        data_dict[prompt_artist]['score1'] = []
        data_dict[prompt_artist]['score2'] = []
        data_dict[prompt_artist]['score3'] = []
        data_dict[prompt_artist]['score1-2'] = []
        data_dict[prompt_artist]['score1-3'] = []

    data_dict[prompt_artist]['score1'].append(score1)
    data_dict[prompt_artist]['score2'].append(score2)
    data_dict[prompt_artist]['score3'].append(score3)
    data_dict[prompt_artist]['score1-2'].append(score1 - score2)
    data_dict[prompt_artist]['score1-3'].append(score1 - score3)

print(len(df_SD))
for prompt_artist in data_dict.keys():
    writestr = "{}: {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}".format(
        prompt_artist,
        np.mean(data_dict[prompt_artist]['score1']), np.std(data_dict[prompt_artist]['score1']),
        np.mean(data_dict[prompt_artist]['score2']), np.std(data_dict[prompt_artist]['score2']),
        np.mean(data_dict[prompt_artist]['score3']), np.std(data_dict[prompt_artist]['score3']),
        np.mean(data_dict[prompt_artist]['score1-2']), np.std(data_dict[prompt_artist]['score1-2']),
        np.mean(data_dict[prompt_artist]['score1-3']), np.std(data_dict[prompt_artist]['score1-3'])
    )
    print(writestr)

1250
tyleredlin: 30.63 \pm 2.22, 23.67 \pm 1.24, 30.12 \pm 2.49, 6.96 \pm 2.37, 0.51 \pm 1.19
thomaskinkade: 34.63 \pm 1.96, 31.13 \pm 2.38, 31.09 \pm 2.22, 3.51 \pm 2.40, 3.54 \pm 1.80
kilianeng: 29.87 \pm 2.64, 25.08 \pm 1.31, 30.54 \pm 2.36, 4.79 \pm 2.35, -0.67 \pm 1.41
kellymckernan: 31.47 \pm 2.58, 27.67 \pm 2.73, 29.69 \pm 2.43, 3.80 \pm 3.46, 1.79 \pm 1.75
ajindemihuman: 30.70 \pm 2.55, 27.65 \pm 3.24, 25.38 \pm 2.77, 3.05 \pm 2.70, 5.32 \pm 3.11
vincentvangogh: 33.66 \pm 2.41, 30.36 \pm 1.17, 28.62 \pm 3.28, 3.29 \pm 2.54, 5.04 \pm 2.96


In [7]:
import math
print('-----------------')
print('To Retain - Same Artist - Cross Models')

for method in method_list:
    df_clip = pd.read_csv(temp_path2.format(method, prompt_name))
    df_lpips = pd.read_csv(temp_path1.format(method, prompt_name))

    print(len(df_clip))
    assert len(df_clip) == len(df_lpips)

    data_dict = dict()
    list_prompt_artist = df_clip['prompt_artist'].unique()
    list_erased_artist = df_clip['erased_artist'].unique()
    print(list_prompt_artist)
    print(list_erased_artist)


    for (_, row_clip), (_, row_lpips) in zip(df_clip.iterrows(), df_lpips.iterrows()):
        prompt = row_clip['prompt']
        prompt_artist = row_clip['prompt_artist']
        erased_artist = row_clip['erased_artist']

        assert row_clip['prompt_artist'] == row_lpips['prompt_artist']
        assert row_clip['erased_artist'] == row_lpips['erased_artist']

        score1 = row_clip['score1']
        score2 = row_clip['score2']
        score3 = row_clip['score3']
        lpips_score = row_lpips['lpips_score']

        if math.isnan(score1) or math.isnan(score2) or math.isnan(score3) or math.isnan(lpips_score):
            break
        
        if prompt_artist not in data_dict:
            data_dict[prompt_artist] = dict()
            data_dict[prompt_artist]['to_erase'] = dict()
            data_dict[prompt_artist]['to_erase']['score1'] = []
            data_dict[prompt_artist]['to_erase']['score2'] = []
            data_dict[prompt_artist]['to_erase']['score3'] = []
            data_dict[prompt_artist]['to_erase']['score1-2'] = []
            data_dict[prompt_artist]['to_erase']['score1-3'] = []
            data_dict[prompt_artist]['to_erase']['lpips_score'] = []

            data_dict[prompt_artist]['to_retain'] = dict()
            data_dict[prompt_artist]['to_retain']['score1'] = []
            data_dict[prompt_artist]['to_retain']['score2'] = []
            data_dict[prompt_artist]['to_retain']['score3'] = []
            data_dict[prompt_artist]['to_retain']['score1-2'] = []
            data_dict[prompt_artist]['to_retain']['score1-3'] = []
            data_dict[prompt_artist]['to_retain']['lpips_score'] = []


        if erased_artist.lower() == prompt_artist:
            data_dict[prompt_artist]['to_erase']['score1'].append(score1)
            data_dict[prompt_artist]['to_erase']['score2'].append(score2)
            data_dict[prompt_artist]['to_erase']['score3'].append(score3)
            data_dict[prompt_artist]['to_erase']['score1-2'].append(score1 - score2)
            data_dict[prompt_artist]['to_erase']['score1-3'].append(score1 - score3)
            data_dict[prompt_artist]['to_erase']['lpips_score'].append(lpips_score)

        else:
            data_dict[prompt_artist]['to_retain']['score1'].append(score1)
            data_dict[prompt_artist]['to_retain']['score2'].append(score2)
            data_dict[prompt_artist]['to_retain']['score3'].append(score3)
            data_dict[prompt_artist]['to_retain']['score1-2'].append(score1 - score2)
            data_dict[prompt_artist]['to_retain']['score1-3'].append(score1 - score3)
            data_dict[prompt_artist]['to_retain']['lpips_score'].append(lpips_score)

    print('--------------')
    print(method)
    for prompt_artist in data_dict.keys():
        for task in ['to_erase', 'to_retain']:
            print(len(data_dict[prompt_artist][task]['score1']))
            writestr = "{}, {}: {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}".format(
                task, prompt_artist,
                np.mean(data_dict[prompt_artist][task]['score1']), np.std(data_dict[prompt_artist][task]['score1']),
                np.mean(data_dict[prompt_artist][task]['score2']), np.std(data_dict[prompt_artist][task]['score2']),
                np.mean(data_dict[prompt_artist][task]['score3']), np.std(data_dict[prompt_artist][task]['score3']),
                np.mean(data_dict[prompt_artist][task]['score1-2']), np.std(data_dict[prompt_artist][task]['score1-2']),
                np.mean(data_dict[prompt_artist][task]['score1-3']), np.std(data_dict[prompt_artist][task]['score1-3']), 
                np.mean(data_dict[prompt_artist][task]['lpips_score']), np.std(data_dict[prompt_artist][task]['lpips_score'])
            )
            print(writestr)
    
    print(' Combine all prompt artist')
    for task in ['to_erase', 'to_retain']:
        print('--------------')
        all_score1 = []
        all_score2 = []
        all_score3 = []
        all_score1_2 = []
        all_score1_3 = []
        all_lpips = []
        for prompt_artist in data_dict.keys():
            all_score1 += data_dict[prompt_artist][task]['score1']
            all_score2 += data_dict[prompt_artist][task]['score2']
            all_score3 += data_dict[prompt_artist][task]['score3']
            all_score1_2 += data_dict[prompt_artist][task]['score1-2']
            all_score1_3 += data_dict[prompt_artist][task]['score1-3']
            all_lpips += data_dict[prompt_artist][task]['lpips_score']
        writestr = "{}, {}: {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}".format(
                task, method,
                np.mean(all_score1), np.std(all_score1),
                np.mean(all_score2), np.std(all_score2),
                np.mean(all_score3), np.std(all_score3),
                np.mean(all_score1_2), np.std(all_score1_2),
                np.mean(all_score1_3), np.std(all_score1_3),
                np.mean(all_lpips), np.std(all_lpips)
            )
        print(writestr)
    print('--------------')


-----------------
To Retain - Same Artist - Cross Models
5000
['tyleredlin' 'thomaskinkade' 'kilianeng' 'kellymckernan' 'ajindemihuman']
['KellyMcKernan' 'KilianEng' 'TylerEdlin' 'ThomasKinkade' 'AjinDemiHuman']
--------------
AP+
200
to_erase, tyleredlin: 23.88 \pm 3.72, 21.92 \pm 1.60, 25.28 \pm 3.56, 1.96 \pm 3.88, -1.40 \pm 1.52, 0.81 \pm 0.13
800
to_retain, tyleredlin: 29.43 \pm 2.84, 22.87 \pm 1.62, 29.53 \pm 2.67, 6.56 \pm 2.90, -0.10 \pm 1.40, 0.44 \pm 0.14
200
to_erase, thomaskinkade: 22.30 \pm 5.20, 18.66 \pm 2.91, 25.38 \pm 4.01, 3.63 \pm 4.38, -3.08 \pm 2.76, 0.78 \pm 0.09
800
to_retain, thomaskinkade: 33.62 \pm 2.75, 29.72 \pm 3.51, 31.00 \pm 2.25, 3.90 \pm 2.65, 2.62 \pm 2.29, 0.41 \pm 0.14
200
to_erase, kilianeng: 21.48 \pm 5.22, 22.30 \pm 1.54, 25.74 \pm 4.42, -0.82 \pm 4.72, -4.25 \pm 2.62, 0.75 \pm 0.09
800
to_retain, kilianeng: 27.68 \pm 3.52, 24.40 \pm 1.44, 28.96 \pm 2.94, 3.28 \pm 3.11, -1.28 \pm 1.79, 0.55 \pm 0.13
200
to_erase, kellymckernan: 20.02 \pm 5.48, 18.

In [8]:
print('-----------------')
print('To Retain - Same Erasure Model - Cross Artists')

for method in method_list:
    df_clip = pd.read_csv(temp_path2.format(method, prompt_name))
    df_lpips = pd.read_csv(temp_path1.format(method, prompt_name))
    
    print(len(df_clip))
    data_dict = dict()
    list_prompt_artist = df_clip['prompt_artist'].unique()
    list_erased_artist = df_clip['erased_artist'].unique()
    print(list_prompt_artist)
    print(list_erased_artist)
    for (_, row_clip), (_, row_lpips) in zip(df_clip.iterrows(), df_lpips.iterrows()):
        prompt = row_clip['prompt']
        prompt_artist = row_clip['prompt_artist']
        erased_artist = row_clip['erased_artist']
        score1 = row_clip['score1']
        score2 = row_clip['score2']
        score3 = row_clip['score3']
        lpips_score = row_lpips['lpips_score']

        if math.isnan(score1) or math.isnan(score2) or math.isnan(score3) or math.isnan(lpips_score):
            continue
        
        if erased_artist not in data_dict:
            data_dict[erased_artist] = dict()
            data_dict[erased_artist]['to_erase'] = dict()
            data_dict[erased_artist]['to_erase']['score1'] = []
            data_dict[erased_artist]['to_erase']['score2'] = []
            data_dict[erased_artist]['to_erase']['score3'] = []
            data_dict[erased_artist]['to_erase']['score1-2'] = []
            data_dict[erased_artist]['to_erase']['score1-3'] = []
            data_dict[erased_artist]['to_erase']['lpips_score'] = []

            data_dict[erased_artist]['to_retain'] = dict()
            data_dict[erased_artist]['to_retain']['score1'] = []
            data_dict[erased_artist]['to_retain']['score2'] = []
            data_dict[erased_artist]['to_retain']['score3'] = []
            data_dict[erased_artist]['to_retain']['score1-2'] = []
            data_dict[erased_artist]['to_retain']['score1-3'] = []
            data_dict[erased_artist]['to_retain']['lpips_score'] = []

        if erased_artist.lower() == prompt_artist:
            data_dict[erased_artist]['to_erase']['score1'].append(score1)
            data_dict[erased_artist]['to_erase']['score2'].append(score2)
            data_dict[erased_artist]['to_erase']['score3'].append(score3)
            data_dict[erased_artist]['to_erase']['score1-2'].append(score1 - score2)
            data_dict[erased_artist]['to_erase']['score1-3'].append(score1 - score3)
            data_dict[erased_artist]['to_erase']['lpips_score'].append(lpips_score)

        else:
            data_dict[erased_artist]['to_retain']['score1'].append(score1)
            data_dict[erased_artist]['to_retain']['score2'].append(score2)
            data_dict[erased_artist]['to_retain']['score3'].append(score3)
            data_dict[erased_artist]['to_retain']['score1-2'].append(score1 - score2)
            data_dict[erased_artist]['to_retain']['score1-3'].append(score1 - score3)
            data_dict[erased_artist]['to_retain']['lpips_score'].append(lpips_score)

    print('--------------')
    print(method)
    for erased_artist in data_dict.keys():
        for task in ['to_erase', 'to_retain']:
            print(len(data_dict[erased_artist][task]['score1']))
            writestr = "{}, {}: {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}".format(
                task, erased_artist,
                np.mean(data_dict[erased_artist][task]['score1']), np.std(data_dict[erased_artist][task]['score1']),
                np.mean(data_dict[erased_artist][task]['score2']), np.std(data_dict[erased_artist][task]['score2']),
                np.mean(data_dict[erased_artist][task]['score3']), np.std(data_dict[erased_artist][task]['score3']),
                np.mean(data_dict[erased_artist][task]['score1-2']), np.std(data_dict[erased_artist][task]['score1-2']),
                np.mean(data_dict[erased_artist][task]['score1-3']), np.std(data_dict[erased_artist][task]['score1-3']), 
                np.mean(data_dict[erased_artist][task]['lpips_score']), np.std(data_dict[erased_artist][task]['lpips_score'])
            )
            print(writestr)
        
    print(' Combine all prompt artist')
    for task in ['to_erase', 'to_retain']:
        print('--------------')
        all_score1 = []
        all_score2 = []
        all_score3 = []
        all_score1_2 = []
        all_score1_3 = []
        all_lpips = []
        for prompt_artist in data_dict.keys():
            all_score1 += data_dict[prompt_artist][task]['score1']
            all_score2 += data_dict[prompt_artist][task]['score2']
            all_score3 += data_dict[prompt_artist][task]['score3']
            all_score1_2 += data_dict[prompt_artist][task]['score1-2']
            all_score1_3 += data_dict[prompt_artist][task]['score1-3']
            all_lpips += data_dict[prompt_artist][task]['lpips_score']
        writestr = "{}, {}: {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}, {:.2f} \pm {:.2f}".format(
                task, method,
                np.mean(all_score1), np.std(all_score1),
                np.mean(all_score2), np.std(all_score2),
                np.mean(all_score3), np.std(all_score3),
                np.mean(all_score1_2), np.std(all_score1_2),
                np.mean(all_score1_3), np.std(all_score1_3),
                np.mean(all_lpips), np.std(all_lpips)
            )
        print(writestr)
    print('--------------')
        

-----------------
To Retain - Same Erasure Model - Cross Artists
5000
['tyleredlin' 'thomaskinkade' 'kilianeng' 'kellymckernan' 'ajindemihuman']
['KellyMcKernan' 'KilianEng' 'TylerEdlin' 'ThomasKinkade' 'AjinDemiHuman']
--------------
AP+
200
to_erase, KellyMcKernan: 20.02 \pm 5.48, 18.57 \pm 4.06, 22.74 \pm 5.15, 1.45 \pm 4.60, -2.73 \pm 2.21, 0.80 \pm 0.11
800
to_retain, KellyMcKernan: 29.97 \pm 3.77, 25.41 \pm 3.79, 28.90 \pm 3.38, 4.56 \pm 3.12, 1.07 \pm 3.22, 0.48 \pm 0.14
200
to_erase, KilianEng: 21.48 \pm 5.22, 22.30 \pm 1.54, 25.74 \pm 4.42, -0.82 \pm 4.72, -4.25 \pm 2.62, 0.75 \pm 0.09
800
to_retain, KilianEng: 30.75 \pm 3.44, 26.33 \pm 3.88, 28.61 \pm 3.38, 4.43 \pm 3.30, 2.14 \pm 3.00, 0.48 \pm 0.13
200
to_erase, TylerEdlin: 23.88 \pm 3.72, 21.92 \pm 1.60, 25.28 \pm 3.56, 1.96 \pm 3.88, -1.40 \pm 1.52, 0.81 \pm 0.13
800
to_retain, TylerEdlin: 29.38 \pm 3.92, 26.65 \pm 3.48, 27.92 \pm 3.56, 2.73 \pm 3.18, 1.47 \pm 3.14, 0.57 \pm 0.12
200
to_erase, ThomasKinkade: 22.30 \pm 5.2

In [9]:
# calculate the mean and std of the lpips score
import math 
for method in method_list:
    data = pd.read_csv(temp_path1.format(method, prompt_name))
    to_erase = []
    to_retain = []
    for _, row in data.iterrows():
        if math.isnan(row['lpips_score']):
            continue
        if row['erased_artist'].lower() == row['prompt_artist']:
            to_erase.append(float(row['lpips_score']))
        else:
            to_retain.append(float(row['lpips_score']))
    # count number of np.nan in to_erase and to_retain
    print(f"method: {method}, to_erase: {len(to_erase)}, {np.mean(to_erase)} +/- {np.std(to_erase)} , to_retain: {len(to_retain)}, {np.mean(to_retain)} +/- {np.std(to_retain)}")
    


method: AP+, to_erase: 1000, 0.7762702849507331 +/- 0.11061882862102253 , to_retain: 4000, 0.48641516115516426 +/- 0.1415254931759478
method: AE, to_erase: 1000, 0.7759636014699935 +/- 0.1052636889348424 , to_retain: 4000, 0.4766424042470753 +/- 0.13671156245852892
